## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Feb. 2nd Feb. 9th


In [3]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,February-6-2025,Condo/Co-op,525 N Ocean Blvd #1419,Pompano Beach,FL,33062.0,440000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/525-N-...,Beaches MLS,F10484227,N,Y,26.240511,-80.088927


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
# df2 = df[(df['SOLD DATE'] > '2024-12-29') & (df['SOLD DATE'] < '2025-01-07')]

In [9]:
df2 = df

In [10]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-02-06    22
2025-02-03    22
2025-02-07    19
2025-02-05    17
2025-02-04    14
2025-02-10     2
Name: count, dtype: int64

In [11]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [12]:
df_top_ten = df2.head(10)

In [13]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
27,PAST SALE,2025-02-04,Condo/Co-op,920 Intracoastal Dr #903,Fort Lauderdale,FL,33304.0,2800000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/920-...,Beaches MLS,F10473897,N,Y,26.135735,-80.109605
42,PAST SALE,2025-02-04,Condo/Co-op,4100 SE Galt Ocean Dr #1512,Fort Lauderdale,FL,33308.0,1850000.0,3.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/4100...,Beaches MLS,F10471394,N,Y,26.178986,-80.097056
100,PAST SALE,2025-02-04,Condo/Co-op,4320 El Mar Dr #202,Lauderdale By The Sea,FL,33308.0,1600000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderdale-by-the-Se...,Beaches MLS,F10387529,N,Y,26.189015,-80.095542
56,PAST SALE,2025-02-03,Condo/Co-op,4240 Galt Ocean Dr #1506,Fort Lauderdale,FL,33308.0,1125000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/4240...,MARMLS,A11680203,N,Y,26.179471,-80.097189
40,PAST SALE,2025-02-05,Condo/Co-op,4445 El Mar Dr Ph 408,Lauderdale By The Sea,FL,33308.0,1037500.0,3.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderdale-By-The-Se...,Beaches MLS,F10473458,N,Y,26.191645,-80.096050
6,PAST SALE,2025-02-03,Condo/Co-op,1578 S Ocean Ln S #114,Fort Lauderdale,FL,33316.0,925000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1578...,MARMLS,A11702861,N,Y,26.103178,-80.106059
35,PAST SALE,2025-02-06,Condo/Co-op,1890 S Ocean Dr Ph 4,Hallandale Beach,FL,33009.0,910000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/189...,MARMLS,A11675223,N,Y,25.982382,-80.118975
84,PAST SALE,2025-02-07,Condo/Co-op,2821 N Ocean Blvd Unit 208S,Fort Lauderdale,FL,33308.0,880000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2821...,Beaches MLS,F10457590,N,Y,26.163462,-80.101526
37,PAST SALE,2025-02-05,Condo/Co-op,333 NE 21st Ave #310,Deerfield Beach,FL,33441.0,760000.0,3.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Deerfield-Beach/333-...,Beaches MLS,F10474798,N,Y,26.318134,-80.076611
5,PAST SALE,2025-02-03,Condo/Co-op,511 SE 5th Ave #1916,Fort Lauderdale,FL,33301.0,755000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/511-...,Beaches MLS,F10477873,N,Y,26.116275,-80.139500


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [14]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [15]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [16]:
response_df = pd.DataFrame(response_list)

In [17]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [18]:
df_top_ten = merged_df

In [19]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2
0,PAST SALE,2025-02-04,Condo/Co-op,920 Intracoastal Dr #903,Fort Lauderdale,FL,33304.0,2800000.0,3.0,3.5,...,-80.109605,https://www.redfin.com/FL/Fort-Lauderdale/920-...,James Cunningham,One Sotheby's Int'l Realty,None,None,James Cunningham,One Sotheby's Int'l Realty,James Cunningham,One Sotheby's Int'l Realty
1,PAST SALE,2025-02-04,Condo/Co-op,4100 SE Galt Ocean Dr #1512,Fort Lauderdale,FL,33308.0,1850000.0,3.0,2.5,...,-80.097056,https://www.redfin.com/FL/Fort-Lauderdale/4100...,Yasminie Iroff,Balistreri Real Estate Inc,None,None,Richard Bonvie,Bonvie Realty,None,None
2,PAST SALE,2025-02-04,Condo/Co-op,4320 El Mar Dr #202,Lauderdale By The Sea,FL,33308.0,1600000.0,3.0,3.5,...,-80.095542,https://www.redfin.com/FL/Lauderdale-by-the-Se...,Richard Masterson,One Sotheby's Int'l Realty,None,None,Howard Goldberg PA,RE/MAX Consultants Realty 1,None,None
3,PAST SALE,2025-02-03,Condo/Co-op,4240 Galt Ocean Dr #1506,Fort Lauderdale,FL,33308.0,1125000.0,3.0,2.0,...,-80.097189,https://www.redfin.com/FL/Fort-Lauderdale/4240...,Danny Shears,Coldwell Banker Realty,None,None,Alfredo Ferro,Envirotek Realty,None,None
4,PAST SALE,2025-02-05,Condo/Co-op,4445 El Mar Dr Ph 408,Lauderdale By The Sea,FL,33308.0,1037500.0,3.0,2.5,...,-80.096050,https://www.redfin.com/FL/Lauderdale-By-The-Se...,Andrew Ziffer,"Compass Florida, LLC",None,None,Ivan Niles,None,None,None
5,PAST SALE,2025-02-03,Condo/Co-op,1578 S Ocean Ln S #114,Fort Lauderdale,FL,33316.0,925000.0,2.0,2.0,...,-80.106059,https://www.redfin.com/FL/Fort-Lauderdale/1578...,Rozita Rafat PA,Keller Williams Legacy,None,None,Rozita Rafat PA,Keller Williams Legacy,None,None
6,PAST SALE,2025-02-06,Condo/Co-op,1890 S Ocean Dr Ph 4,Hallandale Beach,FL,33009.0,910000.0,3.0,3.0,...,-80.118975,https://www.redfin.com/FL/Hallandale-Beach/189...,Vera Aster,Beachfront Realty Inc,None,None,Wendy Newman-Scheppke,Royal Palm Realty Of South Florida,None,None
7,PAST SALE,2025-02-07,Condo/Co-op,2821 N Ocean Blvd Unit 208S,Fort Lauderdale,FL,33308.0,880000.0,2.0,2.0,...,-80.101526,https://www.redfin.com/FL/Fort-Lauderdale/2821...,Kim Garvy,Dettman Realty Group LLC,None,None,Linda Shai,Realty Home Advisors Inc,None,None
8,PAST SALE,2025-02-05,Condo/Co-op,333 NE 21st Ave #310,Deerfield Beach,FL,33441.0,760000.0,3.0,2.5,...,-80.076611,https://www.redfin.com/FL/Deerfield-Beach/333-...,None,None,None,None,None,None,None,None
9,PAST SALE,2025-02-03,Condo/Co-op,511 SE 5th Ave #1916,Fort Lauderdale,FL,33301.0,755000.0,3.0,2.0,...,-80.139500,https://www.redfin.com/FL/Fort-Lauderdale/511-...,Jason Taub,"Taub Real Estate, Inc",None,None,Debra Steinhardt,D'Angelo Realty Group LLC,None,None


## Current Week's Values

In [20]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

104


In [21]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$340,736


In [22]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$270


In [23]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$35,436,550


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [24]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Broward
Input Previous Week Condo Sales Total (number units sold):
118
Input Previous Week Condo Average Sales Price:
424236
Input Previous Week Condo Average PSF:
288
Input Previous Week Condo Sales total (ex: 198_000_000)
50_100_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [25]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [26]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [27]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [28]:
pd.set_option('display.max_columns',None)

In [29]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-02-04,Condo/Co-op,920 Intracoastal Dr #903,Fort Lauderdale,FL,33304.0,2800000.0,3.0,3.5,Aquablu,3258.0,NaN,2018.0,NaN,859.0,4450.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/920-...,Beaches MLS,F10473897,N,Y,26.135735,-80.109605,https://www.redfin.com/FL/Fort-Lauderdale/920-...,James Cunningham,One Sotheby's Int'l Realty,None,None,James Cunningham,One Sotheby's Int'l Realty,James Cunningham,One Sotheby's Int'l Realty,1,orange


In [30]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [31]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-02-04,Condo/Co-op,920 Intracoastal Dr #903,Fort Lauderdale,FL,33304.0,2800000.0,3.0,3.5,Aquablu,3258.0,NaN,2018.0,NaN,859.0,4450.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/920-...,Beaches MLS,F10473897,N,Y,26.135735,-80.109605,https://www.redfin.com/FL/Fort-Lauderdale/920-...,James Cunningham,One Sotheby's Int'l Realty,None,None,James Cunningham,One Sotheby's Int'l Realty,James Cunningham,One Sotheby's Int'l Realty,1,orange
1,PAST SALE,2025-02-04,Condo/Co-op,4100 SE Galt Ocean Dr #1512,Fort Lauderdale,FL,33308.0,1850000.0,3.0,2.5,The Galleon,2900.0,NaN,1967.0,NaN,638.0,2034.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/4100...,Beaches MLS,F10471394,N,Y,26.178986,-80.097056,https://www.redfin.com/FL/Fort-Lauderdale/4100...,Yasminie Iroff,Balistreri Real Estate Inc,None,None,Richard Bonvie,Bonvie Realty,None,None,2,blue
2,PAST SALE,2025-02-04,Condo/Co-op,4320 El Mar Dr #202,Lauderdale By The Sea,FL,33308.0,1600000.0,3.0,3.5,Oriana,3314.0,NaN,2008.0,NaN,483.0,2127.0,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderdale-by-the-Se...,Beaches MLS,F10387529,N,Y,26.189015,-80.095542,https://www.redfin.com/FL/Lauderdale-by-the-Se...,Richard Masterson,One Sotheby's Int'l Realty,None,None,Howard Goldberg PA,RE/MAX Consultants Realty 1,None,None,3,blue
3,PAST SALE,2025-02-03,Condo/Co-op,4240 Galt Ocean Dr #1506,Fort Lauderdale,FL,33308.0,1125000.0,3.0,2.0,LAMBIANCE BEACH CONDO,1836.0,NaN,2003.0,NaN,613.0,1830.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/4240...,MARMLS,A11680203,N,Y,26.179471,-80.097189,https://www.redfin.com/FL/Fort-Lauderdale/4240...,Danny Shears,Coldwell Banker Realty,None,None,Alfredo Ferro,Envirotek Realty,None,None,4,blue
4,PAST SALE,2025-02-05,Condo/Co-op,4445 El Mar Dr Ph 408,Lauderdale By The Sea,FL,33308.0,1037500.0,3.0,2.5,Villas By The Sea,1478.0,NaN,2009.0,NaN,702.0,1495.0,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderdale-By-The-Se...,Beaches MLS,F10473458,N,Y,26.191645,-80.096050,https://www.redfin.com/FL/Lauderdale-By-The-Se...,Andrew Ziffer,"Compass Florida, LLC",None,None,Ivan Niles,None,None,None,5,blue
5,PAST SALE,2025-02-03,Condo/Co-op,1578 S Ocean Ln S #114,Fort Lauderdale,FL,33316.0,925000.0,2.0,2.0,OCEANAGE CONDO,1264.0,NaN,1973.0,NaN,732.0,1275.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1578...,MARMLS,A11702861,N,Y,26.103178,-80.106059,https://www.redfin.com/FL/Fort-Lauderdale/1578...,Rozita Rafat PA,Keller Williams Legacy,None,None,Rozita Rafat PA,Keller Williams Legacy,None,None,6,blue
6,PAST SALE,2025-02-06,Condo/Co-op,1890 S Ocean Dr Ph 4,Hallandale Beach,FL,33009.0,910000.0,3.0,3.0,LA MER ESTATES EAST CONDO,1920.0,NaN,1972.0,NaN,474.0,1732.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/189...,MARMLS,A11675223,N,Y,25.982382,-80.118975,https://www.redfin.com/FL/Hallandale-Beach/189...,Vera Aster,Beachfront Realty Inc,None,None,Wendy Newman-Scheppke,Royal Palm Realty Of South Florida,None,None,7,blue
7,PAST SALE,2025-02-07,Condo/Co-op,2821 N Ocean Blvd Unit 208S,Fort Lauderdale,FL,33308.0,880000.0,2.0,2.0,Sapphire Fort Lauderdale,1500.0,NaN,2009.0,NaN,587.0,1508.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2821...,Beaches MLS,F10457590,N,Y,26.163462,-80.101526,https://www.redfin.com/FL/Fort-Lauderdale/2821...,Kim Garvy,Dettman Realty Group LLC,None,None,Linda Shai,Realty Home Advisors Inc,None,None,8,blue
8,PAST SALE,2025-02-05,Condo/Co-op,333 NE 21st Ave #310,Deerfield Beach,FL,33441.0,760000.0,3.0,2.5,Tiara East Condo,1420.0,NaN,1966.0,NaN,535.0,1507.0,Sold,NaN,Na

In [32]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent {county} County Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [33]:
m.save('index.html')

## Data snagger

In [34]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [35]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-02-04,Condo/Co-op,920 Intracoastal Dr #903,Fort Lauderdale,FL,33304.0,2800000.0,3.0,3.5,Aquablu,3258.0,NaN,2018.0,NaN,859.0,4450.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/920-...,Beaches MLS,F10473897,N,Y,26.135735,-80.109605,https://www.redfin.com/FL/Fort-Lauderdale/920-...,James Cunningham,One Sotheby's Int'l Realty,None,None,James Cunningham,One Sotheby's Int'l Realty,James Cunningham,One Sotheby's Int'l Realty,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [36]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Broward_County_condo_sales_week_ending_02102024


# CREATE TEMPLATE 

In [37]:
muni_set = set(df_top_ten['CITY'])

In [38]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [39]:
df_top_ten.reset_index(inplace=True,drop=True)

In [40]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [41]:
top_sale

'Aquablu at 920 Intracoastal Dr #903 in Fort Lauderdale'

In [42]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [43]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [45]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-02-04,Condo/Co-op,920 Intracoastal Dr #903,Fort Lauderdale,FL,33304.0,2800000.0,3.0,3.5,Aquablu,3258.0,NaN,2018.0,NaN,859.0,4450.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/920-...,Beaches MLS,F10473897,N,Y,26.135735,-80.109605,https://www.redfin.com/FL/Fort-Lauderdale/920-...,James Cunningham,One Sotheby's Int'l Realty,None,None,James Cunningham,One Sotheby's Int'l Realty,James Cunningham,One Sotheby's Int'l Realty,1,orange,Aquablu at 920 Intracoastal Dr #903 in Fort La...
1,PAST SALE,2025-02-04,Condo/Co-op,4100 SE Galt Ocean Dr #1512,Fort Lauderdale,FL,33308.0,1850000.0,3.0,2.5,The Galleon,2900.0,NaN,1967.0,NaN,638.0,2034.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/4100...,Beaches MLS,F10471394,N,Y,26.178986,-80.097056,https://www.redfin.com/FL/Fort-Lauderdale/4100...,Yasminie Iroff,Balistreri Real Estate Inc,None,None,Richard Bonvie,Bonvie Realty,None,None,2,blue,The Galleon at 4100 SE Galt Ocean Dr #1512 in ...
2,PAST SALE,2025-02-04,Condo/Co-op,4320 El Mar Dr #202,Lauderdale By The Sea,FL,33308.0,1600000.0,3.0,3.5,Oriana,3314.0,NaN,2008.0,NaN,483.0,2127.0,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderdale-by-the-Se...,Beaches MLS,F10387529,N,Y,26.189015,-80.095542,https://www.redfin.com/FL/Lauderdale-by-the-Se...,Richard Masterson,One Sotheby's Int'l Realty,None,None,Howard Goldberg PA,RE/MAX Consultants Realty 1,None,None,3,blue,Oriana at 4320 El Mar Dr #202 in Lauderdale By...
3,PAST SALE,2025-02-03,Condo/Co-op,4240 Galt Ocean Dr #1506,Fort Lauderdale,FL,33308.0,1125000.0,3.0,2.0,LAMBIANCE BEACH CONDO,1836.0,NaN,2003.0,NaN,613.0,1830.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/4240...,MARMLS,A11680203,N,Y,26.179471,-80.097189,https://www.redfin.com/FL/Fort-Lauderdale/4240...,Danny Shears,Coldwell Banker Realty,None,None,Alfredo Ferro,Envirotek Realty,None,None,4,blue,LAMBIANCE BEACH CONDO at 4240 Galt Ocean Dr #1...
4,PAST SALE,2025-02-05,Condo/Co-op,4445 El Mar Dr Ph 408,Lauderdale By The Sea,FL,33308.0,1037500.0,3.0,2.5,Villas By The Sea,1478.0,NaN,2009.0,NaN,702.0,1495.0,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderdale-By-The-Se...,Beaches MLS,F10473458,N,Y,26.191645,-80.096050,https://www.redfin.com/FL/Lauderdale-By-The-Se...,Andrew Ziffer,"Compass Florida, LLC",None,None,Ivan Niles,None,None,None,5,blue,Villas By The Sea at 4445 El Mar Dr Ph 408 in ...
5,PAST SALE,2025-02-03,Condo/Co-op,1578 S Ocean Ln S #114,Fort Lauderdale,FL,33316.0,925000.0,2.0,2.0,OCEANAGE CONDO,1264.0,NaN,1973.0,NaN,732.0,1275.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1578...,MARMLS,A11702861,N,Y,26.103178,-80.106059,https://www.redfin.com/FL/Fort-Lauderdale/1578...,Rozita Rafat PA,Keller Williams Legacy,None,None,Rozita Rafat PA,Keller Williams Legacy,None,None,6,blue,OCEANAGE CONDO at 1578 S Ocean Ln S #114 in Fo...
6,PAST SALE,2025-02-06,Condo/Co-op,1890 S Ocean Dr Ph 4,Hallandale Beach,FL,33009.0,910000.0,3.0,3.0,LA MER ESTATES EAST CONDO,1920.0,NaN,1972.0,NaN,474.0,1732.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/189...,MARMLS,A11675223,N,Y,25.982382,-80.118975,https://www.redfin.com/FL/Hallandale-Beach/189...,Vera Aster,Beachfront Realty Inc,None,None,Wendy Newman-Scheppke,Royal Palm Realty Of South Florida,None,None,7,blue,LA MER ESTATES EAST CONDO at 1890 S Ocean Dr P...
7,PAST SALE,2025-02-07,Condo/Co-op,2821 N Ocean Blvd Unit 208S,Fort Lauderdale,FL,33308.0,880000.0,2.0,2.0,Sapphire Fort Lauderdale,1500.0,NaN,2009.0,NaN,587.0,1508.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2821..

In [46]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: Aquablu closing tops Broward weekly condo sales 
DEK: Top 10 sales ranged from $755,000 to $2,800,000
FEATURED HED:
SEO HED: Broward County Weekly Condo Report 
SEO DESCRIPTION: Broward County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Broward condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward, weekly condo sales, Deerfield Beach, Lauderdale By The Sea, Hallandale Beach, Fort Lauderdale

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Broward County’s NEWS PEG HERE.

Brokers closed 104 condo sales totaling $35,436,550 million from Feb. 2nd to Feb. 9th. The previous week, brokers closed 118 condo sales totaling $50,100,000.

Last week’s units sold for an average of $340,736, lower than the $424,236 average price from the previous week. The average price p

In [47]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [48]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [49]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [50]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-02-04,Condo/Co-op,920 Intracoastal Dr #903,Fort Lauderdale,FL,33304.0,2800000.0,3.0,3.5,Aquablu,3258.0,NaN,2018.0,NaN,859.0,4450.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/920-...,Beaches MLS,F10473897,N,Y,26.135735,-80.109605,https://www.redfin.com/FL/Fort-Lauderdale/920-...,James Cunningham,One Sotheby's Int'l Realty,None,None,James Cunningham,One Sotheby's Int'l Realty,James Cunningham,One Sotheby's Int'l Realty,1,orange,Aquablu at 920 Intracoastal Dr #903 in Fort La...
1,PAST SALE,2025-02-04,Condo/Co-op,4100 SE Galt Ocean Dr #1512,Fort Lauderdale,FL,33308.0,1850000.0,3.0,2.5,The Galleon,2900.0,NaN,1967.0,NaN,638.0,2034.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/4100...,Beaches MLS,F10471394,N,Y,26.178986,-80.097056,https://www.redfin.com/FL/Fort-Lauderdale/4100...,Yasminie Iroff,Balistreri Real Estate Inc,None,None,Richard Bonvie,Bonvie Realty,None,None,2,blue,The Galleon at 4100 SE Galt Ocean Dr #1512 in ...
2,PAST SALE,2025-02-04,Condo/Co-op,4320 El Mar Dr #202,Lauderdale By The Sea,FL,33308.0,1600000.0,3.0,3.5,Oriana,3314.0,NaN,2008.0,NaN,483.0,2127.0,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderdale-by-the-Se...,Beaches MLS,F10387529,N,Y,26.189015,-80.095542,https://www.redfin.com/FL/Lauderdale-by-the-Se...,Richard Masterson,One Sotheby's Int'l Realty,None,None,Howard Goldberg PA,RE/MAX Consultants Realty 1,None,None,3,blue,Oriana at 4320 El Mar Dr #202 in Lauderdale By...
3,PAST SALE,2025-02-03,Condo/Co-op,4240 Galt Ocean Dr #1506,Fort Lauderdale,FL,33308.0,1125000.0,3.0,2.0,LAMBIANCE BEACH CONDO,1836.0,NaN,2003.0,NaN,613.0,1830.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/4240...,MARMLS,A11680203,N,Y,26.179471,-80.097189,https://www.redfin.com/FL/Fort-Lauderdale/4240...,Danny Shears,Coldwell Banker Realty,None,None,Alfredo Ferro,Envirotek Realty,None,None,4,blue,LAMBIANCE BEACH CONDO at 4240 Galt Ocean Dr #1...
4,PAST SALE,2025-02-05,Condo/Co-op,4445 El Mar Dr Ph 408,Lauderdale By The Sea,FL,33308.0,1037500.0,3.0,2.5,Villas By The Sea,1478.0,NaN,2009.0,NaN,702.0,1495.0,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderdale-By-The-Se...,Beaches MLS,F10473458,N,Y,26.191645,-80.096050,https://www.redfin.com/FL/Lauderdale-By-The-Se...,Andrew Ziffer,"Compass Florida, LLC",None,None,Ivan Niles,None,None,None,5,blue,Villas By The Sea at 4445 El Mar Dr Ph 408 in ...
5,PAST SALE,2025-02-03,Condo/Co-op,1578 S Ocean Ln S #114,Fort Lauderdale,FL,33316.0,925000.0,2.0,2.0,OCEANAGE CONDO,1264.0,NaN,1973.0,NaN,732.0,1275.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1578...,MARMLS,A11702861,N,Y,26.103178,-80.106059,https://www.redfin.com/FL/Fort-Lauderdale/1578...,Rozita Rafat PA,Keller Williams Legacy,None,None,Rozita Rafat PA,Keller Williams Legacy,None,None,6,blue,OCEANAGE CONDO at 1578 S Ocean Ln S #114 in Fo...
6,PAST SALE,2025-02-06,Condo/Co-op,1890 S Ocean Dr Ph 4,Hallandale Beach,FL,33009.0,910000.0,3.0,3.0,LA MER ESTATES EAST CONDO,1920.0,NaN,1972.0,NaN,474.0,1732.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/189...,MARMLS,A11675223,N,Y,25.982382,-80.118975,https://www.redfin.com/FL/Hallandale-Beach/189...,Vera Aster,Beachfront Realty Inc,None,None,Wendy Newman-Scheppke,Royal Palm Realty Of South Florida,None,None,7,blue,LA MER ESTATES EAST CONDO at 1890 S Ocean Dr P...
7,PAST SALE,2025-02-07,Condo/Co-op,2821 N Ocean Blvd Unit 208S,Fort Lauderdale,FL,33308.0,880000.0,2.0,2.0,Sapphire Fort Lauderdale,1500.0,NaN,2009.0,NaN,587.0,1508.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2821..

In [59]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: Aquablu closing tops Broward weekly condo sales 
DEK: Top 10 sales ranged from $755,000 to $2,800,000
FEATURED HED:
SEO HED: Broward County Weekly Condo Report 
SEO DESCRIPTION: Broward County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Broward condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward, weekly condo sales, Deerfield Beach, Lauderdale By The Sea, Hallandale Beach, Fort Lauderdale

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Broward County’s NEWS PEG HERE.

Brokers closed 104 condo sales totaling $35,436,550 million from Feb. 2nd to Feb. 9th. The previous week, brokers closed 118 condo sales totaling $50,100,000.

Last week’s units sold for an average of $340,736, lower than the $424,236 average price from the previous week. The average price p

In [52]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Fort-Lauderdale/920-Intracoastal-Dr-33304/unit-903/home/113703787


In [53]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Fort-Lauderdale/4100-Galt-Ocean-Dr-33308/unit-1512/home/41707709


In [54]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Fort-Lauderdale/511-SE-5th-Ave-33301/unit-1916/home/42065073


In [55]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Deerfield-Beach/333-NE-21st-Ave-33441/unit-310/home/41506208


In [56]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Fort-Lauderdale/920-Intracoastal-Dr-33304/unit-903/home/113703787


In [57]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Fort-Lauderdale/511-SE-5th-Ave-33301/unit-1916/home/42065073


In [58]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-02-04,Condo/Co-op,920 Intracoastal Dr #903,Fort Lauderdale,FL,33304.0,2800000.0,3.0,3.5,Aquablu,3258.0,NaN,2018.0,NaN,859.0,4450.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/920-...,Beaches MLS,F10473897,N,Y,26.135735,-80.109605,https://www.redfin.com/FL/Fort-Lauderdale/920-...,James Cunningham,One Sotheby's Int'l Realty,None,None,James Cunningham,One Sotheby's Int'l Realty,James Cunningham,One Sotheby's Int'l Realty,1,orange,Aquablu at 920 Intracoastal Dr #903 in Fort La...
1,PAST SALE,2025-02-04,Condo/Co-op,4100 SE Galt Ocean Dr #1512,Fort Lauderdale,FL,33308.0,1850000.0,3.0,2.5,The Galleon,2900.0,NaN,1967.0,NaN,638.0,2034.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/4100...,Beaches MLS,F10471394,N,Y,26.178986,-80.097056,https://www.redfin.com/FL/Fort-Lauderdale/4100...,Yasminie Iroff,Balistreri Real Estate Inc,None,None,Richard Bonvie,Bonvie Realty,None,None,2,blue,The Galleon at 4100 SE Galt Ocean Dr #1512 in ...
2,PAST SALE,2025-02-04,Condo/Co-op,4320 El Mar Dr #202,Lauderdale By The Sea,FL,33308.0,1600000.0,3.0,3.5,Oriana,3314.0,NaN,2008.0,NaN,483.0,2127.0,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderdale-by-the-Se...,Beaches MLS,F10387529,N,Y,26.189015,-80.095542,https://www.redfin.com/FL/Lauderdale-by-the-Se...,Richard Masterson,One Sotheby's Int'l Realty,None,None,Howard Goldberg PA,RE/MAX Consultants Realty 1,None,None,3,blue,Oriana at 4320 El Mar Dr #202 in Lauderdale By...
3,PAST SALE,2025-02-03,Condo/Co-op,4240 Galt Ocean Dr #1506,Fort Lauderdale,FL,33308.0,1125000.0,3.0,2.0,LAMBIANCE BEACH CONDO,1836.0,NaN,2003.0,NaN,613.0,1830.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/4240...,MARMLS,A11680203,N,Y,26.179471,-80.097189,https://www.redfin.com/FL/Fort-Lauderdale/4240...,Danny Shears,Coldwell Banker Realty,None,None,Alfredo Ferro,Envirotek Realty,None,None,4,blue,LAMBIANCE BEACH CONDO at 4240 Galt Ocean Dr #1...
4,PAST SALE,2025-02-05,Condo/Co-op,4445 El Mar Dr Ph 408,Lauderdale By The Sea,FL,33308.0,1037500.0,3.0,2.5,Villas By The Sea,1478.0,NaN,2009.0,NaN,702.0,1495.0,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderdale-By-The-Se...,Beaches MLS,F10473458,N,Y,26.191645,-80.096050,https://www.redfin.com/FL/Lauderdale-By-The-Se...,Andrew Ziffer,"Compass Florida, LLC",None,None,Ivan Niles,None,None,None,5,blue,Villas By The Sea at 4445 El Mar Dr Ph 408 in ...
5,PAST SALE,2025-02-03,Condo/Co-op,1578 S Ocean Ln S #114,Fort Lauderdale,FL,33316.0,925000.0,2.0,2.0,OCEANAGE CONDO,1264.0,NaN,1973.0,NaN,732.0,1275.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1578...,MARMLS,A11702861,N,Y,26.103178,-80.106059,https://www.redfin.com/FL/Fort-Lauderdale/1578...,Rozita Rafat PA,Keller Williams Legacy,None,None,Rozita Rafat PA,Keller Williams Legacy,None,None,6,blue,OCEANAGE CONDO at 1578 S Ocean Ln S #114 in Fo...
6,PAST SALE,2025-02-06,Condo/Co-op,1890 S Ocean Dr Ph 4,Hallandale Beach,FL,33009.0,910000.0,3.0,3.0,LA MER ESTATES EAST CONDO,1920.0,NaN,1972.0,NaN,474.0,1732.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/189...,MARMLS,A11675223,N,Y,25.982382,-80.118975,https://www.redfin.com/FL/Hallandale-Beach/189...,Vera Aster,Beachfront Realty Inc,None,None,Wendy Newman-Scheppke,Royal Palm Realty Of South Florida,None,None,7,blue,LA MER ESTATES EAST CONDO at 1890 S Ocean Dr P...
7,PAST SALE,2025-02-07,Condo/Co-op,2821 N Ocean Blvd Unit 208S,Fort Lauderdale,FL,33308.0,880000.0,2.0,2.0,Sapphire Fort Lauderdale,1500.0,NaN,2009.0,NaN,587.0,1508.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2821..

## Time on Market Calculator

In [61]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 11, 26) ## List (Earlier) date
date2 = datetime(2025, 2, 4) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

70


## Clean CSV for Datawrapper Chart

In [62]:
chart_df = df_top_ten

In [63]:
chart_df = chart_df.fillna(" ")

In [64]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [65]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [66]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [67]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [68]:
chart_df['$/SQUARE FEET'].astype(float)

0    859.0
1    638.0
2    483.0
3    613.0
4    702.0
5    732.0
6    474.0
7    587.0
8    535.0
9    446.0
Name: $/SQUARE FEET, dtype: float64

In [69]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [70]:
csv_date_string = today.strftime("%m_%d_%Y")

In [71]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")

In [72]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2025-02-04,920 Intracoastal Dr #903 in Fort Lauderdale,"$2,800,000",3,3.5,"3,258",2018,$859,Beaches MLS,F10473897,James Cunningham One Sotheby's Int'l Realty,James Cunningham One Sotheby's Int'l Realty Ja...
1,2025-02-04,4100 SE Galt Ocean Dr #1512 in Fort Lauderdale,"$1,850,000",3,2.5,"2,900",1967,$638,Beaches MLS,F10471394,Yasminie Iroff Balistreri Real Estate Inc,Richard Bonvie Bonvie Realty
2,2025-02-04,4320 El Mar Dr #202 in Lauderdale By The Sea,"$1,600,000",3,3.5,"3,314",2008,$483,Beaches MLS,F10387529,Richard Masterson One Sotheby's Int'l Realty,Howard Goldberg PA RE/MAX Consultants Realty 1...
3,2025-02-03,4240 Galt Ocean Dr #1506 in Fort Lauderdale,"$1,125,000",3,2.0,"1,836",2003,$613,MARMLS,A11680203,Danny Shears Coldwell Banker Realty,Alfredo Ferro Envirotek Realty
4,2025-02-05,4445 El Mar Dr Ph 408 in Lauderdale By The Sea,"$1,037,500",3,2.5,"1,478",2009,$702,Beaches MLS,F10473458,"Andrew Ziffer Compass Florida, LLC",Ivan Niles
5,2025-02-03,1578 S Ocean Ln S #114 in Fort Lauderdale,"$925,000",2,2.0,"1,264",1973,$732,MARMLS,A11702861,Rozita Rafat PA Keller Williams Legacy,Rozita Rafat PA Keller Williams Legacy
6,2025-02-06,1890 S Ocean Dr Ph 4 in Hallandale Beach,"$910,000",3,3.0,"1,920",1972,$474,MARMLS,A11675223,Vera Aster Beachfront Realty Inc,Wendy Newman-Scheppke Royal Palm Realty Of Sou...
7,2025-02-07,2821 N Ocean Blvd Unit 208S in Fort Lauderdale,"$880,000",2,2.0,"1,500",2009,$587,Beaches MLS,F10457590,Kim Garvy Dettman Realty Group LLC,Linda Shai Realty Home Advisors Inc
8,2025-02-05,333 NE 21st Ave #310 in Deerfield Beach,"$760,000",3,2.5,"1,420",1966,$535,Beaches MLS,F10474798,,
9,2025-02-03,511 SE 5th Ave #1916 in Fort Lauderdale,"$755,000",3,2.0,"1,692",2006,$446,Beaches MLS,F10477873,"Jason Taub Taub Real Estate, Inc",Debra Steinhardt D'Angelo Realty Group LLC
